In [27]:
import pickle
import json
import requests
# import numpy as np
import pandas as pd
from datetime import datetime, timezone
from kafka import KafkaProducer, KafkaConsumer
# from sklearn.linear_model import Lasso

In [28]:
number_of_entries = 300
link = f"https://api.cryptobay.io/bay/cryptobaygetauctionlist?\
data=%7B%22object%22%3A%7B%7D%2C%22token_type%22%3A1%2C%22state%22%3A%220%22%2C%22\
order_by%22%3A%222%22%2C%22page_num%22%3A0%2C%22page_size%22%3A{number_of_entries}%7D"

try:
    relative_location
except NameError:
    relative_location = ""

r = requests.get(link)

raw_json = r.json()

In [29]:
# KAFKA setup

with open(f"{relative_location}tools/zzsecrets.json") as f:
    data = json.load(f)
    CLOUDKARAFKA_USERNAME = data['CLOUDKARAFKA_USERNAME']
    CLOUDKARAFKA_PASSWORD = data['CLOUDKARAFKA_PASSWORD']

CLOUDKARAFKA_BROKERS="glider-01.srvs.cloudkafka.com:9094,glider-02.srvs.cloudkafka.com:9094,glider-03.srvs.cloudkafka.com:9094"
CLOUDKARAFKA_TOPIC_PREFIX=f"{CLOUDKARAFKA_USERNAME}-"

kafka_topic="predict"
kafka_topic = CLOUDKARAFKA_TOPIC_PREFIX+kafka_topic


In [30]:
producer = KafkaProducer(bootstrap_servers=CLOUDKARAFKA_BROKERS,
                         sasl_plain_username=CLOUDKARAFKA_USERNAME,
                         sasl_plain_password=CLOUDKARAFKA_PASSWORD,
                         security_protocol='SASL_SSL',
                         sasl_mechanism='SCRAM-SHA-256',
                         ssl_cafile=f"{relative_location}tools/cafile.crt",
                        #  api_version=(0,10),
                         retries=5,
                         )

In [31]:
def get_binance_price_coinograph() -> int:
    """
    Example
            {
        "exchange": "binance",
        "pair": "bnbusdt",
        "price": 335.8,
        "ask": 335.8,
        "bid": 335.7
        }
    """
    price_json = requests.get("https://coinograph.io/ticker/?symbol=binance:bnbusdt").json()
    return price_json['price']


In [32]:
def get_binance_price_coingecko() -> float:
    """
    Example
{'id': 'binancecoin',
 'symbol': 'bnb',
 'name': 'Binance Coin',
 'image': 'https://assets.coingecko.com/coins/images/825/large/binance-coin-logo.png?1547034615',
 'current_price': 400.07,
 'market_cap': 61889077351,
 'market_cap_rank': 5,
 'fully_diluted_valuation': 68296906490,
 'total_volume': 1565273152,
 'high_24h': 401.01,
 'low_24h': 372.01,
 'price_change_24h': 25.8,
 'price_change_percentage_24h': 6.89444,
 'market_cap_change_24h': 3991379311,
 'market_cap_change_percentage_24h': 6.89385,
 'circulating_supply': 154533651.9,
 'total_supply': 170533651.9,
 'max_supply': 170533651.9,
 'ath': 686.31,
 'ath_change_percentage': -41.64571,
 'ath_date': '2021-05-10T07:24:17.097Z',
 'atl': 0.0398177,
 'atl_change_percentage': 1005707.34064,
 'atl_date': '2017-10-19T00:00:00.000Z',
 'roi': None,
 'last_updated': '2021-10-01T08:23:03.665Z'}
    """
    url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=binancecoin"
    price_json = requests.get(url).json()[0]['current_price']
    return price_json

In [33]:
def get_ship_stats_selling(bnb_price: float):
    ship_id_list = pd.DataFrame([element['token_id'] for element in raw_json['data']['auctions']])
    tranzaction_id_list = pd.DataFrame([element['idx'] for element in raw_json['data']['auctions']])
    ship_class_list = pd.DataFrame([element['class'] for element in raw_json['data']['auctions']])
    ship_price_list_BNB = pd.DataFrame([ round(int(element['buyout_price']) / 1000000000, 3) for element in raw_json['data']['auctions'] ])
    ship_price_list_USD = pd.DataFrame([ round(int(element['buyout_price']) / 1000000000 * bnb_price, 2) for element in raw_json['data']['auctions'] ])

    boat_space_list = pd.DataFrame([ element['raw_space'] for element in raw_json['data']['auctions']])
    boat_speed_list = pd.DataFrame([ element['raw_speed'] for element in raw_json['data']['auctions']])
    boat_skill_list = pd.DataFrame([ element['raw_skill'] for element in raw_json['data']['auctions']])
    boat_defence_list = pd.DataFrame([ element['raw_defence'] for element in raw_json['data']['auctions']])
    boat_attack_list = pd.DataFrame([ element['raw_attack'] for element in raw_json['data']['auctions']])
    boat_morale_list = pd.DataFrame([ element['raw_morale'] for element in raw_json['data']['auctions']])
    
    concat_DF = pd.concat([ship_id_list, tranzaction_id_list, ship_class_list, \
                        boat_space_list, boat_speed_list, boat_skill_list, \
                        boat_defence_list, boat_attack_list, boat_morale_list,\
                        ship_price_list_USD, ship_price_list_BNB], axis=1)
    
    concat_DF.columns
    
    return concat_DF

In [34]:
# IMPORT MODELS
model_USD = pickle.load(open(f"{relative_location}db/model_USD.sav", 'rb'))
model_BNB = pickle.load(open(f"{relative_location}db/model_BNB.sav", 'rb'))

# zztest = np.array([5,13,15,6,11,5])
# zztest = zztest.reshape(1, -1)

# model_BNB.predict(zztest)


In [35]:
columns = ['transaction_id', 'ship_id', 'ship_class', 'ship_durability', 'ship_owner', \
        'ship_attr_space', 'ship_attr_speed', 'ship_attr_skill', 'ship_attr_defence', 'ship_attr_attack', 'ship_attr_morale', \
        'ship_parts_keel', 'ship_parts_sail', 'ship_parts_side', 'ship_parts_bow', 'ship_parts_cabin', 'ship_parts_stern', \
        'ship_sold_price']

In [36]:
#ship_price_list = [ round(int(element['buyout_price']) / 1000000000 * bnb_price, 2) for element in raw_json['data']['auctions']]

In [37]:
type(get_binance_price_coingecko())

float

In [38]:
try:
    bnb_price
except NameError:
    bnb_price = get_binance_price_coingecko()

# Force refresh for bnb_price 
#bnb_price = get_binance_price_coingecko()

stats_df = get_ship_stats_selling(bnb_price)
stats_df.columns = ['ship_id', 'transaction_id', 'ship_class','ship_attr_space', \
    'ship_attr_speed', 'ship_attr_skill', 'ship_attr_defence', \
    'ship_attr_attack', 'ship_attr_morale', 'selling_price_USD', 'selling_price_BNB']
#stats_df.head(2)

In [39]:
pred_columns = ['ship_attr_space', 'ship_attr_speed', 'ship_attr_skill', \
                'ship_attr_defence', 'ship_attr_attack', 'ship_attr_morale']

X = stats_df[pred_columns]

#X.head()

In [40]:
y_pred_USD = model_USD.predict(X)
y_pred_USD = pd.Series(y_pred_USD)
y_pred_USD = y_pred_USD.round(2)

In [41]:
y_pred_BNB = model_BNB.predict(X)
y_pred_BNB = pd.Series(y_pred_BNB)
y_pred_BNB = y_pred_BNB.round(3)

In [42]:
_df = pd.concat([stats_df, y_pred_USD, y_pred_BNB], axis=1)
_df.rename(columns={0: 'predicted_price_USD', 1: 'predicted_price_BNB'}, inplace=True)
_df['flip_ratio'] = _df['predicted_price_BNB'] / _df['selling_price_BNB']
_df['flip_ratio'] = _df['flip_ratio'].round(2)
gold_df = _df[['ship_id', 'transaction_id', 'selling_price_USD', 'predicted_price_USD', 'selling_price_BNB', 'predicted_price_BNB', 'flip_ratio']]

In [53]:
gold_df.head()

,ship_id,transaction_id,selling_price_USD,predicted_price_USD,selling_price_BNB,predicted_price_BNB,flip_ratio
0,14162,20124,154.82,44.17,0.35,0.125,0.36
1,4214,20123,53.08,32.74,0.12,0.088,0.73
2,8164,20122,132.70,72.51,0.30,0.219,0.73
3,14375,20120,221.17,47.11,0.50,0.133,0.27
4,52498,20118,128.28,90.86,0.29,0.285,0.98


In [57]:
final_df = (gold_df[gold_df['flip_ratio'] > 1.1]\
    .sort_values('flip_ratio', ascending=False))

In [45]:
final_df.to_csv(f"{relative_location}db/out/predicted_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M')}.csv", header=False, index=False)

In [58]:
final_df

,ship_id,transaction_id,selling_price_USD,predicted_price_USD,selling_price_BNB,predicted_price_BNB,flip_ratio
232,18031,19476,88.47,73.94,0.20,0.223,1.12
7,8780,20115,66.35,56.44,0.15,0.167,1.11


In [47]:
###
to_kafka_df = final_df.to_csv(header=False, index=False)

In [48]:

producer.send(kafka_topic, bytes(f"{to_kafka_df}", encoding='utf8'))

In [49]:
print(kafka_topic+"\n")
print(to_kafka_df)

2b581ilu-predict

18031,19476,88.47,73.94,0.2,0.223,1.12
8780,20115,66.35,56.44,0.15,0.167,1.11
24349,19876,88.47,70.71,0.2,0.211,1.06
44883,19632,88.47,71.23,0.2,0.209,1.04
17152,20008,88.47,68.79,0.2,0.205,1.02
18483,19524,88.47,66.99,0.2,0.203,1.01
7470,19941,92.89,71.63,0.21,0.212,1.01
17365,20117,92.89,71.15,0.21,0.213,1.01
6946,19637,110.58,82.03,0.25,0.247,0.99
52498,20118,128.28,90.86,0.29,0.285,0.98
16206,19899,92.89,67.62,0.21,0.204,0.97
15238,19339,88.47,63.94,0.2,0.186,0.93
9563,20067,88.47,60.71,0.2,0.184,0.92
28322,19190,97.31,69.16,0.22,0.202,0.92
11577,20086,110.58,76.28,0.25,0.227,0.91
13212,19352,88.47,61.49,0.2,0.183,0.91
16654,19704,88.47,61.18,0.2,0.182,0.91
17797,19547,92.89,62.64,0.21,0.188,0.9
366,19663,88.47,59.77,0.2,0.179,0.89
10284,19987,88.47,60.66,0.2,0.178,0.89
6817,19799,88.47,59.99,0.2,0.178,0.89
24693,19496,79.18,56.19,0.179,0.16,0.89
35717,19838,88.47,60.89,0.2,0.178,0.89
381,19708,88.47,60.44,0.2,0.178,0.89
11181,19907,79.62,52.85,0.18,0.158,0.88
229

In [50]:
# cols = [ col for col in _df.columns if 'attr' in col]
# _df[cols].astype('float').sum(axis=1).sort_values(ascending=False)


In [51]:
# zztest = _df.iloc[83].values[3:9]

# zztest = np.array([5,13,15,5,7,12])
# zztest = np.array([5,13,11,8,9,13])
# zztest = np.array(zztest).reshape(1, -1)
# model_USD.predict(zztest)
# model_BNB.predict(zztest)